In [1]:
import numpy as np
import torch, pickle
import lightgbm as lgb
from hummingbird import convert_sklearn
from onnxconverter_common.data_types import FloatTensorType

In [2]:

# load the LGBM model for Fraud
model = pickle.load(open('lgbm-500-6.pkl', "rb"))
#model = lgb.LGBMClassifier(n_estimators=10, max_depth=10)
X = np.random.rand(100000, 28)
X = np.array(X, dtype=np.float32)
y = np.random.randint(3, size=100000)

model.fit(X, y)

pytorch_model = convert_sklearn(
    model, 
    [("input", FloatTensorType([100000, 28]))], 
    extra_config={"tree_implementation": "perf_tree_trav"})

X_torch = torch.from_numpy(X)


In [7]:
%%timeit -r 3

#time for skl
skl = model.predict_proba(X)

4.61 s ± 47.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [8]:
%%timeit -r 3

# time for hummingbird - CPU
pytorch_model.to('cpu')
hum_cpu = pytorch_model(X_torch)

7.97 s ± 29.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [9]:
%%timeit -r 3

# time for hummingbird - GPU. Note that you must have a GPU-enabled machine.
pytorch_model.to('cuda')
hum_gpu = pytorch_model(X_torch.to('cuda'))

255 ms ± 66.5 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [10]:
# make sure Hummingbird output matches Scikit-learn as expected
skl = model.predict_proba(X)
pytorch_model.to('cuda')
hum_gpu = pytorch_model(X_torch.to('cuda'))

np.testing.assert_allclose(skl, hum_gpu[1].data.to('cpu').numpy(), rtol=1e-06, atol=1e-06)